<a href="https://colab.research.google.com/github/shan2312/EVA4/blob/master/S4/EVA4_Session_2_SS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

torch.manual_seed(1)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=0) # RF - 3, O/p - 26
        self.drop1 = nn.Dropout(0.1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=0) # RF - 5, O/p - 24
        self.pool1 = nn.MaxPool2d(2, 2) # RF - 10, 12, O/p - 12 
        self.conv3 = nn.Conv2d(32, 8, 1) # RF - 10, 12, O/p - 12
        self.conv4 = nn.Conv2d(8, 16, 3, padding=0) # RF - 14, O/p - 10
        self.conv5 = nn.Conv2d(16, 32, 3, padding=1) # RF - 14, O/p - 10
        self.pool2 = nn.MaxPool2d(2, 2) # RV - 28, 5, O/p - 5
        self.conv6 = nn.Conv2d(32, 16, 1) # RF 28, 5, O/p - 5
        self.conv7 = nn.Conv2d(16, 32, 3, padding=0) # RF- 30, 3
        self.conv8 = nn.Conv2d(32, 10, 3, padding=0) # RF- - 32, 1
      
        
    def forward(self, x):
        x = self.pool1(self.drop1(F.relu(self.conv2(self.drop1(F.relu(self.conv1(x)))))))
        x = self.pool2(self.drop1(F.relu(self.conv5(self.drop1(F.relu(self.conv4(self.drop1(F.relu(self.conv3(x))))))))))
        x = self.conv8(self.drop1(F.relu(self.conv7(self.drop1(F.relu(self.conv6(x)))))))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [21]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
           Dropout-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 32, 24, 24]           4,640
           Dropout-4           [-1, 32, 24, 24]               0
         MaxPool2d-5           [-1, 32, 12, 12]               0
            Conv2d-6            [-1, 8, 12, 12]             264
           Dropout-7            [-1, 8, 12, 12]               0
            Conv2d-8           [-1, 16, 10, 10]           1,168
           Dropout-9           [-1, 16, 10, 10]               0
           Conv2d-10           [-1, 32, 10, 10]           4,640
          Dropout-11           [-1, 32, 10, 10]               0
        MaxPool2d-12             [-1, 32, 5, 5]               0
           Conv2d-13             [-1, 16, 5, 5]             528
          Dropout-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [24]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0905, Accuracy: 9751/10000 (97.51%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0546, Accuracy: 9823/10000 (98.23%)



  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0193, Accuracy: 9936/10000 (99.36%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0213, Accuracy: 9935/10000 (99.35%)



loss=0.00448945164680481 batch_id=937: 100%|██████████| 938/938 [00:20<00:00, 46.76it/s]



Test set: Average loss: 0.0192, Accuracy: 9947/10000 (99.47%)

